<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2020.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 1: Pixel-wise operations and the Fourier transform</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday October 1, 2020</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Friday October 9, 2020</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weigth</b>: 9% of the overall grade</p>
    <p style="margin:4px;"><b>Remote help</b>: Monday October 5, on Zoom (see Moodle for link and time)</p>    
    <p style="margin:4px;"><b>Related lectures</b>: Chapters 1 and 2</p>
</div>

### Student Name: Filip Slezak
### SCIPER: 286557

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [127]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

SCIPER: 286557


### <a name="imports_"></a> Imports
In the next cell we import the python libraries that we will use throughout the lab, as well as the `IPLabViewer()` class (see documentation [here](https://github.com/Biomedical-Imaging-Group/IPLabImageViewer/wiki/Python-IPLabViewer()-Class), or run the python command `help(viewer)` after loading the class):
* [`matplotlib.pyplot`](https://matplotlib.org), to display images,
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive, and
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays.

Finally, we load the images used in this lab.

Run the cell to get your notebook ready.

<div class=" alert alert-danger">
    
**Note:** Always run the two import cells below before starting to work on the notebook.
    
</div>

In [128]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import required packages for this lab
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

# Loading IPLabViewer 
from lib.iplabs import IPLabViewer as viewer

# Loading images
hrct = plt.imread( "images/hrct.tif" )
joux = plt.imread( "images/joux.tif" )
car = plt.imread( "images/car_pad.tif" )
mandrill = plt.imread( "images/mandrill.tif")
impulse = np.zeros((65,65)); impulse[32,32] = 1;
pens = plt.imread( "images/pens.tif" )
zebra = plt.imread("images/zebra.tif")

In the following cell we import the `IPLabImageAccess` class. You can find the documentation of the class [here](https://github.com/Biomedical-Imaging-Group/IPLabImageAccess/wiki).

In [129]:
%use javascript
// import the IPLabImageAccess class as Image
var Image = require('./lib/IPLabImageAccess.js')

# Pixel-wise operations and the Fourier transform (14 points)

In the first part of this lab you will learn the basics of pixel-wise image processing by performing simple operations. For example, you will learn to use color to see more details in images. 

In the second part we will look at the 2D discrete Fourier transform, what it represents, and how an image can be reconstructed from its 2D discrete Fourier transform by using the inverse discrete Fourier transform.

## <a class="anchor"></a> Index
1. [Pixel-wise operations](#-1.-Pixel-wise-operations-(8-Points))
    1. [16-bit gray-scale images: Visualization and Colorization](#-1.A.-16-bit-gray-scale-images:-Visualization-and-Colorization-(3-Points)) **(3 points)**
    2. [Image normalization](#-1.B.-Image-normalization-(6-Points)) **(6 points)**
2. [Understanding the Fourier transform](#-2.-Understanding-the-Fourier-transform-(5-Points))
    1. [The FT and its inverse](#-2.A.-The-FT-and-its-inverse-(3-Points)) **(3 points)**
    2. [Reconstruction](#-2.B.-Reconstruction-(2-Points)) **(2 points)**
    

<div class=" alert alert-danger">
    
<b>Important:</b> Each cell that contains code begins with `%use sos` or `%use javascript`. This indicates if the code in this specific cell should be executed in Python or JavaScript. Do not change or remove any lines of code that begin with an %. They need to be on the first line of each cell!
    
</div>

Take some time to explore the images you will be using. Run the next cell and look at the histograms, the range of values, etc.
<div class="alert alert-info">
    
**Hint:** Use the buttons `Prev`/`Next` to cycle through the images.
</div>

In [130]:
%use sos
# Define the list of images
image_list = [hrct, joux, car, mandrill, impulse, pens, zebra]
# Display all images used in this lab
initial_viewer = viewer(image_list, hist = True)

# <a class="anchor"></a> 1. Pixel-wise operations (8 Points) 
## <a class="anchor"></a> 1.A. 16-bit gray-scale images: Visualization and Colorization (3 Points)
[Back to index](#-Index)
### Limitations in visualization

Most standard screens use only 8 bits to visualize gray-level images, which allows for $2^8 = 256$ different gray-levels to be displayed at the same time. For day-to-day photography this is enough, but to store all the important information contained in medical images, a 16-bit representation is often necessary, which provides $2^{16} = 65536$ different gray-levels. This poses a problem because a standard screen cannot show all the gray-levels in a 16-bit image at the same time. To illustrate the problem we will look at the image `hrct`, which is encoded with 16 bits. The image shows the result of a [computed tomography](https://en.wikipedia.org/wiki/CT_scan) scan of a human thorax. These type of images can, for example, be used to diagnose or assess the developement of COVID-19 in patients ([see more here](https://radiologyassistant.nl/chest/covid-19/covid19-imaging-findings)). 

Run the next cell and explore the gray-level range. Try to find hidden content in the image that is not visible at first (at first you will only see the thorax). 
<div class="alert alert-info">
    
**Hint:** You can adjust the gray-level range of the image by adjusting the values (max / min) of the <i>Brightness & Contrast</i> slider.
</div>

In [131]:
%use sos
# Display the hrct image to find hidden information
plt.close("all")
hrct_viewer = viewer(hrct, title = 'HRCT', widgets = True)

### Multiple choice question
For **1 point**, once you have explored the image, answer the next questions:

* Q1: How many 8-bit grayscale images do we need to store the same information contained in a generic 16-bit image?
    1. 2,
    2. 8,
    3. 256, or
    4. 16.


* Q2: As you will study below, using color is another option to see more of the information contained in a 16-bit image on screen. Now, choose `Options` and select the colormap `nipy_spectral` (make sure that the *Brightness & Contrast* slider spans the whole range). This view reveals wide-spread structures within the patient's lungs, which could be relevant to doctors. Which of the following ranges with a `gray` colormap show those details best?
    1. $0\%$ to $10\%$,
    2. $90\%$ to $100\%$, or
    3. $50\%$ to $60\%$.

Modify the variables `answer_one` and `answer_two` in the next cell to reflect your choice.

In [132]:
%use sos
# Assign your answer to this variable
answer_one = 3
answer_two = 1
# YOUR CODE HERE

In [133]:
%use sos
# Check that the answer is in the correct range
assert answer_one in [1,2,3,4], 'Possible values are 1, 2, 3 or 4.'

In [134]:
%use sos
# Check that the answer is in the correct range
assert answer_two in [1,2,3], 'Possible values are 1, 2, or 3.'

### <a class="anchor"></a> Colorization

As we have seen, then, color is a powerful tool to overcome the representation limitations of 8-bit screens. The basic idea is to express the 16-bit range $[0,65535]$ using a combination of three 8-bit $[0, 255]$ channels to make a color (RGB) image. This exercise will guide you through this process.

There are many alternatives to divide a range into sub-ranges (_a.k.a._ colormaps), but you will implement the one specified by the picture below:

<img src="images/graylevel_divide_rgb.png" alt="Drawing" style="width: 500px;"/>

The three lines reflect the intensity values of each channel depending on the original graylevel intensity. $N$ is the maximum graylevel intensity for a 16-bit image, i.e., ($2^{16} - 1$).

For **1 point**, modify the function `color_pixel_wise(img)` in the next cell to iterate through every pixel in the image and create a new colorized image. The three color channels should be defined according to the figure above.
<div class="alert alert-info">

**Note:** You only need to modify the variables `r`, `g` and `b`. Everything else has already been prepared for you. Make sure you understand the code and fill in the blanks.
</div>

In [135]:
%use javascript
// function that divides a 16 bit graylevel image into a rgb image with 8 bits per channel
function color_pixel_wise(img){
    // the max value of the original image (16 bits)
    var N = Math.pow(2,16) - 1;
    // the max value of each channel of the new image (8 bits)
    var new_N = 255;
    // initialize output image (color image)
    options = {}; options.rgb = true;
    var output = new Image(img.ny, img.nx, options);
    // Iterate through each pixel
    for(var x = 0; x < img.nx; x++){
        for(var y = 0; y < img.ny; y++){
            // initialize the red, green and blue channels to 0
            var r = 0, g = 0, b = 0;
            // get the pixel value at the current location
            var value = img.getPixel(x, y);
            
            // assign the correct values to the red, green and blue channels according to the proposed mapping
            // YOUR CODE HERE
            if(value < 0.25*N) {
                r = 1020/N*value;
            } else if(value < 0.50*N) {
                g = 1020/N*(value - 0.25*N);
                r = 255 - g;
            } else if(value < 0.75*N) {
                b = 1020/N*(value - 0.50*N);
                g = 255 - b;
            } else {
                b = 255 - 1020/N*(value - 0.75*N);
            }
            // set the three color channels in the output image (convert them to integers using Math.round)
            output.setPixel(x, y, [Math.round(r), Math.round(g), Math.round(b)])
        }
    }
    return output;
}

Great, using the two cells below we are going to visualize your image. If everything went well, you should see most of the hidden details in red (blood vessels and details inside the lungs), the middle values in green (soft tissues, fat, etc.) and the higher values in blue (aorta, bone tissue, etc.). 

In the next cell, we are first going to get the image `hrct` from the Python kernel, and then apply your function to it. Finally, we put the result (`hrct_colorized_js`) back into the Python kernel for visualization. 

In [136]:
%use javascript
%get hrct
%put hrct_colorized_js

var hrct_img = new Image(hrct);
var hrct_colorized_js = color_pixel_wise(hrct_img).toArray();

Now that your result is stored in Python, run the next cell to visualize it.

<div class="alert alert-info">
    
<b>Note: </b> SoS translates JS arrays as Python lists. However, `IPLabViewer` (and every major IP library) works with NumPy arrays, so we have to explicitly call the method <code>np.array()</code> on the result from JS.
</div>

In [137]:
%use sos
# Convert to NumPy array
hrct_colorized_js = np.array(hrct_colorized_js)
# Visualize
plt.close('all')
hrct_colorized_viewer = viewer(hrct_colorized_js, title='HRTC colorized JS')

Button(description='Show Widgets', style=ButtonStyle())

Is your result correct? At the top of the image you should see the color bar transition **smoothly** from black to red, from red to green, from green to blue, and back to black again.

Let's make some sanity tests to check that the image is really a color image and that the maximum and minimum value of each channel are $0$ and $255$ respectively. We will constantly be making these kinds of tests to ensure the correct behaviour of any code you write.

In [138]:
%use sos
# Check that the image has indeed 3 color channels
assert hrct_colorized_js.shape[2] == 3, "The resulting image doesn't have 3 color channels!"
# Check that the max and min of each channel are 0 and 255
assert np.min(hrct_colorized_js[:,:,0]) == 0, f"The minimum of the red color channel is {np.min(hrct_colorized_js[:,:,0])} and not 0!"
assert np.min(hrct_colorized_js[:,:,1]) == 0, f"The minimum of the green color channel is {np.min(hrct_colorized_js[:,:,1])} and not 0!"
assert np.min(hrct_colorized_js[:,:,2]) == 0, f"The minimum of the blue color channel is {np.min(hrct_colorized_js[:,:,2])} and not 0!"
assert np.max(hrct_colorized_js[:,:,0]) == 255, f"The maximum of the red color channel is {np.max(hrct_colorized_js[:,:,0])} and not 255!"
assert np.max(hrct_colorized_js[:,:,1]) == 255, f"The maximum of the green color channel is {np.max(hrct_colorized_js[:,:,1])} and not 255!"
assert np.max(hrct_colorized_js[:,:,2]) == 255, f"The maximum of the blue color channel is {np.max(hrct_colorized_js[:,:,2])} and not 255!"
print("Congrats, your function passed the sanity checks. However, that does not necessarily mean that everything is correct.")

Congrats, your function passed the sanity checks. However, that does not necessarily mean that everything is correct.


If we wanted to perform the same colorization in Python, we shouldn't do it pixel-wise as in JavaScript, since this will be extremely slow. As you saw in the introductory lab, languages that allow __[vectorization](https://en.wikipedia.org/wiki/Automatic_vectorization)__ like Python (through the NumPy library) and MATLAB allow operations to be performed on whole arrays. This is much simpler to code and as fast as a `for` loop in a low-level programming language. 

In the following cell, we show you how to colorize your image in Python, without the need to iterate through every pixel. This function is exactly like the one you created in JavaScript, but we added the parameters `peak1`, `peak2` and `peak3`, which will be useful in the next exercise. Make sure to fully understand the function. If you have any doubts, go back to the [Introductory lab](./Introductory.ipynb), or read about [boolean indexing in NumPy](https://numpy.org/devdocs/reference/arrays.indexing.html#boolean-array-indexing).

When you run the next cell, you will generate and visualize the variable `hrct_colorized_python`. It should look exactly like the result you got from the JavaScript function above. 

In [139]:
%use sos
# Function that divides a `bits` bit graylevel image into a rgb image with 8 bits per channel depending on the three specified limits (in %)
def color_vectorized(img, peak1 = 25, peak2 = 50, peak3 = 75, bits = 16):     
    # Initialize max value
    N = 2**bits - 1
    N_new = 2**8 - 1
    
    # Make sure that peak1 < peak2 < peak3
    peak1, peak2, peak3 = np.sort([peak1, peak2, peak3])
    
    # Adjust limit values from percent to absolute value
    peak1 = peak1 / 100 * N
    peak2 = peak2 / 100 * N
    peak3 = peak3 / 100 * N
    
    # Initialize 3 color channels with the appropriate dimensions
    color_R = np.zeros(img.shape)
    color_G = np.zeros(img.shape)
    color_B = np.zeros(img.shape)
    
    # Generate boolean arrays corresponding to the 4 different sections. 
    section_1 = img < peak1
    section_2 = np.logical_and(peak1 <= img, img < peak2)
    section_3 = np.logical_and(peak2 <= img, img < peak3)
    section_4 = peak3 <= img
    
    # Assign the pixel values of each channel depending on the section
    color_R[section_1] = img[section_1] / peak1
    color_R[section_2] = 1 - (img[section_2] - peak1) / (peak2 - peak1)
    color_G[section_2] = (img[section_2] - peak1) / (peak2 - peak1)
    color_G[section_3] = 1 - (img[section_3] - peak2) / (peak3 - peak2)
    color_B[section_3] = (img[section_3] - peak2) / (peak3 - peak2)
    color_B[section_4] = 1 - (img[section_4] - peak3) / (N - peak3)
    
    # Concatenate the three color channels into one color image    
    color_img = np.dstack((color_R,color_G,color_B))
    # Multiply by maximum and round to 8-bit integers
    color_img = np.round(color_img * N_new).astype(np.uint8)

    return(color_img)


# Run the function for the same (evenly spaced) limits as in the JavaScript function
hrct_colorized_python = color_vectorized(hrct)

# Visualize results
plt.close('all')
hrct_colorized_python_viewer = viewer([hrct_colorized_js, hrct_colorized_python], 
                                      title=['JS colorization', 'Python colorizaiton'], subplots=(1,2)) 

Button(description='Show Widgets', style=ButtonStyle())

Besides the visual test, you can use the next cell to compare your implementation to our function in Python. The results should be nearly identical. We use the function numpy [`assert_array_almost_equal`](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_almost_equal.html), and compare the two arrays up to the 4th decimal number. Run the next cell, if it runs smoothly, your implementation is correct.

In [140]:
%use sos
# Test if the two images are almost the same
np.testing.assert_array_almost_equal(hrct_colorized_python, hrct_colorized_js, decimal = 4, err_msg='Hint: Check your your range limits')
print('Good job! Your implementation in JS was correct.')

Good job! Your implementation in JS was correct.


As you can see, this is already a much better representation than only using one gray-level channel. However, we can do even better: In the above image, we used evenly spaced peaks for the triangles in the color mapping. However, if we know that some intensity ranges of the image contain more information than others, we can adjust the peaks to increase the visibility of these intensity ranges.

In the cell below we will add an extra functionality to the `viewer`: We will declare three sliders that will allow us to dynamically set the peaks of the triangles in our colorization, as well as the corresponding button and activation function (to review how this works, check the introductory lab). The activation function will get the value of the sliders, and call the method `color_vectorized()` on the input image.

Below the viewer, we will plot a histogram of the image, overlayed with the triangles implemented by `color_vectorized()` for each slider selection. In this histogram the peaks will update in real time as you move the sliders in the viewer. This is meant for you to get a better idea of which ranges of values in the image enclose more information, and how one should design a colorization to reveal this information. **You do not need to understand this code-block**, however, if you are curious about `matplotlib` and `ipywidgets`, take the time to understand what we are doing.

Run the next cell and click on the button *Extra Widgets* to play with the three sliders. To show the effect of the current slider selection, click on `Apply Colorization`.

In [141]:
%use sos
plt.close('all')

# Defining the sliders and the button of the extra widget
peak1_slider = widgets.IntSlider(value = 25, min = 0, max = 100, step = 1, description = 'Peak 1 (%)')
peak2_slider = widgets.IntSlider(value = 50, min = 0, max = 100, step = 1, description = 'Peak 2 (%)')
peak3_slider = widgets.IntSlider(value = 75, min = 0, max = 100, step = 1, description = 'Peak 3 (%)')
activation_button = widgets.Button(description = 'Apply Colorization')

# Sort sliders whenever a user crosses them
def sort_sliders():
    peak1_slider.value, peak2_slider.value, peak3_slider.value = np.sort([peak1_slider.value, 
                                                                          peak2_slider.value, 
                                                                          peak3_slider.value])
# Defining the callback function of the button
def activation_callback(img):
    # Sort sliders (should not be necessary)
    sort_sliders()
    # Colorize image
    output = color_vectorized(img, peak1_slider.value, peak2_slider.value, peak3_slider.value)
    return output

# Visualize the image with the extra widget functionality
colorization_ranges_viewer = viewer(hrct, title = 'Personalizing your colormap', 
                                    new_widgets = [peak1_slider, peak2_slider, peak3_slider, activation_button], 
                                    callbacks = [activation_callback], widgets=True)

## The code below plots the interactive histogram.
## Feel free to explore it, but without any pressure.

# Maximum value in image
N = np.amax(hrct)
# Compute the histogram
hist, bins = np.histogram(hrct, bins = 70, range = (0, N))
# 10% over maximum number of counts (arbitrary max value for triangles)
Y = 1.1*np.amax(hist)
# Declare a matplotlib figure, capture its axes, plot the histogram, select axis' limits, and set x ticks to %
fig = plt.figure(num=f"SCIPER: {uid}",figsize = (4, 2.7)); ax = plt.gca()
ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) / 1.2)
ax.set_xlim(0, N); ax.set_ylim(0, Y); plt.yticks([],[])
plt.xticks([0,.25*N,.5*N,.75*N,N],[r"$0\%$",r"$25\%$",r"$50\%$",r"$75\%$",r"$100\%$"])
plt.title("Image Histogram and Colorization")
# List to store the lines that will form the triangles
lines = []
# Function that generates the triangles based on the peaks' positions
def generate_triangles(peak1, peak2, peak3):
    peak1, peak2, peak3 = np.sort([peak1, peak2, peak3])
    xdata = [[0, peak1], [peak1, peak2], [peak1, peak2], [peak2, peak3] , [peak2, peak3], [peak3, N]]
    ydata = [[0, Y], [Y, 0], [0, Y], [Y, 0] , [0, Y], [Y, 0]]
    return xdata, ydata

# Initial plot of the lines that form the triangles (2 per triangle)
color = 2*'r' + 2*'g'+ 2*'b' 
for i in range(6):
    xdata, ydata = generate_triangles(N/4, N/2, 3*N/4)
    lines.append(ax.plot(xdata[i], ydata[i], color[i]))
    
# Callback of sliders
def update_histogram(change):
    sort_sliders()  
    # Get the data 
    xdata, _ = generate_triangles(N*peak1_slider.value/100, N*peak2_slider.value/100, N*peak3_slider.value/100)
    # Update lines
    count = 0    
    for line in lines:
        line[0].set_xdata(xdata[count])
        count += 1
        
# Link sliders to callback (the three to the same callback)
for slider in [peak1_slider, peak2_slider, peak3_slider]:
    slider.observe(update_histogram, 'value') 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

### Multiple choice question
For **1 point**, which of the following combination of peaks allows you to see the most information?
<div class="alert alert-info">
    
<b>Tip:</b> If you do not see the sliders' values, slide right or set your browser's magnification to $100\%$.
</div>

1. `Peak 1` = $10\%$, `Peak 2` = $30\%$, `Peak 3` = $60\%$,
2. `Peak 1` = $25\%$, `Peak 2` = $50\%$, `Peak 3` = $75\%$, or
3. `Peak 1` = $10\%$, `Peak 2` = $60\%$, `Peak 3` = $85\%$.

Modify the variable `answer` in the next cell to reflect your answer.

In [142]:
%use sos
# Assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [143]:
%use sos
# Check that the answer is in the valid range
assert answer in [1, 2, 3], 'Possible answers are 1, 2 or 3.'

##  <a class="anchor"></a> 1.B. Image normalization (6 Points)
[Back to index](#-Index)

In this section you will learn different ways to normalize an image.

**For a total of 3 points**, your assignment is to complete the three functions below (**1 point each**), which output images normalized with respect to different statistics. In particular, you have to complete:
* `makeZeroMean(img)`: Normalizes the image so that the sample mean of the pixel values is zero.
* `stretchContrast(img)`: Normalizes the image so that the minimum value is $0$ and the maximum value is $1$. 
* `normalize2ndOrderStatistics(img)`: Normalizes the image so that the sample mean of the pixel values is zero and the sample standard deviation is $1$. 

JS, unlike Python, **does not** have a function to calculate the mean or the standard deviation. Thus, you will need to code them yourself explicitly.
<div class="alert alert-info">
    
<b>Hint: </b> You can use `img.getMin()` and `img.getMax()` to get the min an max value of a JavaScript image `img`. Remeber also that you can access a wide range of mathematical functions through the Math library in JS (e.g., `Math.sqrt()`). You can read more about it [here](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Math).
</div>

First, implement the method `makeZeroMean` in the cell below.

In [144]:
%use javascript
// function that normalizes the image so that the sample mean of the pixel values is zero.
function makeZeroMean(img){
    // declare the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    var mean = 0;
    for(var i = 0; i < img.nx; ++i) {
        for(var j = 0; j < img.ny; ++j) {
            mean += img.getPixel(i, j);
        }
    }
    mean /= img.nx*img.ny;
    for(var i = 0; i < img.nx; ++i) {
        for(var j = 0; j < img.ny; ++j) {
            output.setPixel(i, j, img.getPixel(i, j) - mean);
        }
    }    
    // return the output image
    return output;
}

Great! Now it's time to test your implementation. A partial test is to run the next cell, which will test your method on a simple $3\times 3$ array. If an error is thrown, you implementation is not yet correct.   

In [145]:
%use javascript

// declare the test image
test_img = new Image([[0, 1, 2], [3, 4, 5], [6, 7, 8]]);

// run the zero mean function
var test_zero_mean = makeZeroMean(test_img);

// check if the output is as expected 
if(!(test_zero_mean.imageCompare(new Image([[-4, -3, -2], [-1, 0, 1], [2, 3, 4]])))){
        throw new Error("makeZeroMean() is not yet correct");
}
// print victory message
console.log('Nice, the function seems to be correct!');

Nice, the function seems to be correct!


Now, implement `stretchContrast`.

In [146]:
%use javascript

// function that normalizes the image so that all pixels have values between 0 and 1.
function stretchContrast(img){
    // declare the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    for(var i = 0; i < img.nx; ++i) {
        for(var j = 0; j < img.ny; ++j) {
            output.setPixel(i, j, (img.getPixel(i, j) - img.getMin())/(img.getMax() - img.getMin()));
        }
    }
    // return the output image
    return output;
}

Run the next cell to test the function.

In [147]:
%use javascript

// run the stretch contrast function on the test image
var test_stretch = stretchContrast(test_img);

// compare the result to the correct result
if(!(test_stretch.imageCompare(new Image([[0, 0.125, 0.25], [0.375, 0.5, 0.625], [0.75, 0.875, 1]])))){
    throw new Error("Stretch Contrast not yet correct");
}
// print victory message
console.log('Nice, the function seems to be correct!');

Nice, the function seems to be correct!


Finally, implement the function `normalize2ndOrderStatistics`.

In [148]:
%use javascript

// function that normalizes the image so that the sample mean of the pixel values is 0 and the sample standard deviation is 1.
function normalize2ndOrderStatistics(img){
    // declare the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    output = makeZeroMean(img);
    //console.log(output);
    var mean = 0;
    for(var i = 0; i < img.nx; ++i) {
        for(var j = 0; j < img.ny; ++j) {
            mean += img.getPixel(i, j);
        }
    }
    mean /= img.nx*img.ny;
    var std = 0;
    for(var i = 0; i < img.nx; ++i) {
        for(var j = 0; j < img.ny; ++j) {
            std += Math.pow(img.getPixel(i, j) - mean, 2);
        }
    }
    std = Math.sqrt(std/(img.nx*img.ny));
    for(var i = 0; i < img.nx; ++i) {
        for(var j = 0; j < img.ny; ++j) {
            output.setPixel(i, j, output.getPixel(i, j)/std);
        }
    }    
    // return the output image
    return output;
}

And run the next cell for a quick test again.

In [149]:
%use javascript

// run the function on test_img
var test_normalize = normalize2ndOrderStatistics(test_img).toArray();

// compare the result to the correct result
// test for unbiased estimator of variance
if(!(Image.arrayCompare(test_normalize, [[ -1.4605934866804429, -1.0954451150103321, -0.7302967433402214 ],
                                         [ -0.3651483716701107, 0, 0.3651483716701107 ],
                                         [ 0.7302967433402214, 1.0954451150103321, 1.4605934866804429 ]]))){
    // test for biased estimator of variance
    if(!(Image.arrayCompare(test_normalize, [[ -1.5491933384829668, -1.161895003862225, -0.7745966692414834 ],
                                             [ -0.3872983346207417, 0, 0.3872983346207417 ],
                                             [ 0.7745966692414834, 1.161895003862225, 1.5491933384829668 ]]))){
        throw new Error("Normalize 2nd Order Statistics not yet correct");
    }else{
        // print victory message
        console.log('Nice, the function seems to be correct! You\'re using the biased estimator of the variance.');
    }
}else{
    // print victory message
    console.log('Nice, the function seems to be correct! You\'re using the unbiased estimator of the variance.');
}


Nice, the function seems to be correct! You're using the biased estimator of the variance.


#### In order for you to see the relevance of image normalization, we provide a sequence of fluorescence microscopy images (see more [here](https://en.wikipedia.org/wiki/Fluorescence_microscope)), named `c_elegans`. These are consecutive slices of the same 3D volume, but appear darker every time due to photobleaching (the loss of flourescence, read more [here](https://en.wikipedia.org/wiki/Photobleaching)). Of course, this is a huge problem for the application, and to solve it we absolutely need to normalize these images.

Here, we will use the module [io](https://scikit-image.org/docs/0.8.0/api/skimage.io.html) of SciKit-Image, which allows us to read all the slices of a `.tif` file at once. Run the next cell to load the `c_elegans` images, display them, see a graph of the effect of photobleaching on their mean value, and pass them to JS so that we can use the functions you defined above. Make sure you explore the images and their histogram by clicking on the `Prev` and `Next` buttons. 

In [150]:
%use sos
%put c_elegans --to javascript

# We import module io to import tif images as slices
from skimage import io
# The following cell loads the image c-elegans
c_elegans = io.imread( "images/c-elegans.tif" ) 
# Show c_elegans images\
plt.close("all")
viewer([c_elegans[ind,:,:] for ind in range(12)], normalize = False, title=[f"c_elegans {ind+1}" for ind in range(12)], hist=True)
# Show decay of the mean value through time due to photobleaching
fig = plt.figure(num=f"SCIPER: {uid}",figsize = (6, 4))
plt.plot([ind+1 for ind in range(12)], [np.mean(c_elegans[ind,:,:]) for ind in range(12)])
plt.xticks([ind+1 for ind in range(12)]); plt.xlabel("Image number"); plt.ylabel("Mean value"); 
plt.grid('both'); plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

Now we are going to visualize the effect of each of your normalizing functions on the `c_elegans` images. For this purpose, we provide you the function `make_montage(img_arr, mode, cols)`, which takes as parameters:
* `img_arr` (array): an array of `Image` objects, 
* `mode` (1, 2 or 3): the function to apply (1: zero mean, 2: stretch contrast, 3: normalize statistics), and
* `cols` (int):, the number of columns to use in the montage. 

The function first creates an empy montage `out`. `out` has the dimensions to fit all the images given in `img_arr`. The method then performs the operation specified by `mode` on each image, placing the result in the right place inside the montage `out`. 

Now run the next cell to declare the function `makeMontage()`.

In [151]:
%use javascript

// function that creates a single image from multiple images (slices) and performs the specified function on the images
function makeMontage(img_arr, mode, cols) {
    // get dimensions of each image
    var w = img_arr[0].nx;
    var h = img_arr[0].ny;
    // determine the number of rows 
    var rows = img_arr.length/cols;
    // initialize output image 
    var out = new Image(h*rows, w*cols);    
    // iterate through each image in img_arr
    for(t=0; t<rows*cols; t++){
        // extract the corresponding image 
        var img = img_arr[t].copy();
        // check requested operation 
        //(note that any mode other than 1, 2 or 3 simply copies the original images in the montage)
        if(mode == 1){ 
            img = makeZeroMean(img);
        }             
        if(mode == 2){
            img = stretchContrast(img);
        }
        if(mode == 3){
            img = normalize2ndOrderStatistics(img);
        }            
        // put result in the corresponding place
        out.putSubImage(Math.floor(t%cols)*w, Math.floor(t/cols)*h, img);
    }
    // return output image
    return out;
}

Now, we are going apply your methods to the image slices we just loaded. First, we convert each element in the array `c_elegans`  to an `Image` object. Then, we call the function on the array with each of the three modes to visualize the result of the functions that you implemented above.

In [152]:
%use javascript
%put montage_original montage_zero_mean_js montage_normalize_statistics_js montage_stretch_contrast_js 

// convert each element in the c_elegans array to an Image object
var c_elegans_imgs = new Array();
for(x = 0; x < c_elegans.length; x++){
    c_elegans_imgs.push(new Image(c_elegans[x]));
}

// run makeMontage with all four functions (modes)
var montage_original = makeMontage(c_elegans_imgs, 0, 3).toArray();
var montage_zero_mean_js = makeMontage(c_elegans_imgs, 1, 3).toArray();
var montage_stretch_contrast_js = makeMontage(c_elegans_imgs, 2, 3).toArray();
var montage_normalize_statistics_js = makeMontage(c_elegans_imgs, 3, 3).toArray();

12

Now that we have applied your functions and that we have the variables in Python, let's visualize them. Run the following cell to do so. Use the buttons `Next` and `Prev` to browse through the tree images. If your implementations passed the previous tests, you should see the correct result.

In [153]:
%use sos
# Define the lists of images and titles
image_list = [np.array(montage_original), np.array(montage_zero_mean_js), np.array(montage_stretch_contrast_js), np.array(montage_normalize_statistics_js)]
title_list = ['Original c_elegans', 'Zero Mean c_elegans', 'Stretch Contrast c_elegans', 'Normalize Statistics c_elegans']

# Display the montages
plt.close('all')
normalization_viewer = viewer(image_list, title = title_list, widgets=True)

### Multiple choice questions

Why does the bottom-right corner of the **Zero Mean Montage** have lower contrast than the top-left corner? (**0.5 points**)

1. Because the different subimages in the montage have different spreads of values around their mean.
2. It is an illumination effect.
3. The bottom-right subimages of the montage are defective.

Modify the variable `answer` in the next cell to reflect your choices. As usual, there is another cell that will remind you to select a valid choice if you haven't.

In [154]:
%use sos

# Assign your answer to this variable
answer = 1
# YOUR CODE HERE

In [155]:
%use sos

assert answer in [1, 2, 3], 'Possible answers are 1, 2 or 3' 

Why is this not the case for the other two montages? (**0.5 points**)

1. Becuase the images already had zero mean.
2. Because the other two functions modify the contrast by adjusting the range of intensities.

Modify the variable `answer` in the next cell to reflect your choices. As usual, there is another cell that will remind you to select a valid choice if you haven't.

In [156]:
%use sos

# assign your answer to this variable
answer = 2
# YOUR CODE HERE

In [157]:
%use sos

assert answer in [1, 2], 'Possible answers are 1 and 2'

As you have probably realized by now, low level languages can get tedious. You can do the same thing you did in JavaScript in less lines in Python, by using NumPy arrays, so let's do it! 

We give you the method `make_zero_mean`. **For a total of 2 points**, implement the other two methods (`stretch_contrast` and `normalize_2nd_order_statistics`) in Python in the following cell (**1 point each**). When appropriate, use the following functions:
- `np.mean(img)` returns the estimated mean value of `img`,
- `np.min(img)` and `np.max(img)` return the min and max of `img` respectively,
- `np.std(img)` returns the estimated standard deviation of `img`, based on the biased estimator of the variance (see more about the `ddof` parameter running `help(np.std)`).

<div class="alert alert-info">

**Hint:** 
- If you're unsure how to handle numpy arrays, look at Section 2.A.a of the [Introductory lab](./Introductory.ipynb),
- Only one line of code needs to be filled in for every function, but do not worry if you prefer to use more lines.
</div>

In [158]:
%use sos
# Function that normalizes the image so that the sample mean of the pixel values is zero.
def make_zero_mean(img):
    # Declare the output image
    output = np.copy(img)
    # Subtract the mean from the input image
    output = img-np.mean(img)
    # Return the output image
    return output 

# Function that normalizes the image so that all pixels have values between 0 and 1.
def stretch_contrast(img):
    # Declare the output image
    output = np.copy(img)
    
    # YOUR CODE HERE
    output = (output - np.min(output)) / (np.max(output) - np.min(output))
    # Return the output image
    return output

# Function that normalizes the image so that the sample mean of the pixel values is 0 and the sample standard deviation is 1.
def normalize_2nd_order_statistics(img):
    # Declare the output image
    output = np.copy(img)
    
    # YOUR CODE HERE
    output = (output - np.mean(output)) / np.std(output)
    # Return the output image
    return output

Use the next two cells for a quick test on your functions. This cell tests the two characteristics requested for each function:
* that the result of `stretch_contrast` is in the range $[0, 1]$, and
* that the result of `normalize_2nd_order_statistics` has zero mean and unit variance. 

Run them, and if your implementations are correct, they shouldn't raise any errors.

In [159]:
%use sos

# This cell tests your method stretch contrast
# Here we run your function on the first slice of c_elegans
test_stretch_contrast = stretch_contrast(c_elegans[0])

# And we check that stretch_contrast effectively maps the pixels to the range [0,1]
assert np.min(test_stretch_contrast) == 0, 'The minimum value in the result of stretch_contrast is not 0'
assert np.max(test_stretch_contrast) == 1, 'The maximum value in the result of stretch_contrast is not 1'
print("Well done! Your stretch_contrast function seems to be correct.")


Well done! Your stretch_contrast function seems to be correct.


In [160]:
%use sos

# This cell tests your method normalize statistics
# Here we run the method on the first slice of c_elegans
test_normalize_statistics = normalize_2nd_order_statistics(c_elegans[0])

# Now we check that normalize_statistics returns an image with mean = 0, 
assert np.abs(np.mean(test_normalize_statistics)) < 1e-10, 'Your mean in normalize_2nd_order_statistics is not 0'
# And with std = 1 
assert np.abs(np.std(test_normalize_statistics) - 1) < 1e-4 or np.abs(np.std(test_normalize_statistics, ddof=1) - 1) < 1e-5, 'Your standard deviation in normalize_2nd_order_statistics is not 1'
print('Well done! Your normalization of 2nd order statistics seems to be correct.')


Well done! Your normalization of 2nd order statistics seems to be correct.


# <a class="anchor"></a> 2. Understanding the Fourier transform (5 Points)
[Back to index](#-Index)

This section is dedicated to 1) understanding the effects of the elements in an image on its Fourier transform (FT), and 2) understanding how an image is reconstructed from its FT using the inverse Fourier transform (IFT). From here on we will only use Python, since implementing a FT in a low level language is beyond the scope of this course. To compute the FT in Python, we will use the [`fft` module](https://numpy.org/doc/stable/reference/routines.fft.html) in NumPy, which implements the FT using a [fast Fourier transform (FFT)](https://en.wikipedia.org/wiki/Fast_Fourier_transform) algorithm.

## <a class="anchor"></a> 2.A. The FT and its inverse (3 Points)
[Back to index](#-Index)

First, we will provide the functions `fourier(img)` and `inverse_fourier(ft)`, which calculate the FT and the IFT respectively.

<div class="alert alert-info">

**Note:** 
- `np.fft.fft2(img)` calculates the two-dimensional FT of `img`,
- `np.fft.fftshift(ft)` shifts the frequency range of the FT `ft` from $[0, \pi]$ to $[-\frac{\pi}{2}, \frac{\pi}{2}]$,
- `np.fft.ifft2(ft)` calculates the inverse FT of the two-dimensional FT `ft`.
</div>

In [161]:
%use sos

# Function that returns the FT
def fourier(img):
    # Generate the FT
    ft = np.fft.fft2(img)
    # Shift the frequency range to [-pi/2, pi/2] 
    shift_ft = np.fft.fftshift(ft)
    return shift_ft

# Function that return the inverse FT
def inverse_fourier(ft):
    # Shift the FT back to [0, pi]
    ft = np.fft.ifftshift(ft)
    # Get the inverse FT
    ift = np.fft.ifft2(ft)
    # Clip the imaginary part of the reconstruction
    # (should be approximately zero anyway)
    ift = np.real(ift)
    return ift

As you know, calculating the FT of an image generates a two-dimensional array (image) of complex values, which makes it challenging to find a good visualization. Therefore, we usually extract the **magnitude** and **phase** of the complex numbers, which are much easier to deal with and present useful information. Remember that the magnitude of a complex number $z\in\mathbb{C}$ is given by
$$|z| = \sqrt{\operatorname{Re}(z)^2+\operatorname{Im}(z)^2}.$$

Further, we usually want to visualize this magnitude in dB, i.e., 

$$|z|~[\mathrm{dB}] = 10\log_{10}\left(|z|^2\right).$$

One of the reasons for this is that the variations in the magnitude of the Fourier transform generally span very different ranges, from the very small to the very large, and the $\log(\cdot)$ transformation allows us to visualize both in the same image.

To do this, we will first define the function `magnitude(ft)`, which should return the magnitude in decibels (dB) of a FT given as an input parameter. For **0.5 points**, complete the function `magnitude(ft)` in the cell below according to the equation given above.

<div class="alert alert-info">

**Hints:**
- With NumPy, you can extract the **real** part of a complex variable `z` using [`np.real(z)`](https://numpy.org/doc/stable/reference/generated/numpy.real.html) and the **imaginary** part using [`np.imag(z)`](https://numpy.org/doc/stable/reference/generated/numpy.imag.html),
- if you need it, use [`np.sqrt(x)`](https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html) to get the square root of `x`,
- use [`np.log10(x)`](https://numpy.org/doc/stable/reference/generated/numpy.log10.html) to get the base-10 logarithm of `x`.
</div>

<div class="alert alert-warning">

**Beware:** Using `np.absolute` in this exercise will give you **0 points**! Implement the function yourself.
</div>

In [162]:
%use sos

# Function that returns the magnitude of the FT in dB
def magnitude(ft):
    # Initialize the output to 0
    output = 0
    
    # YOUR CODE HERE
    output = 10*np.log10(np.real(ft)**2 + np.imag(ft)**2)
    # Return the output
    return output

In [163]:
# Let's do a sanity check
# The complex number used for the test which has a magnitude of ~3 dB
z = 1 + 1j
# Check that the magnitude function is correct
assert np.round(magnitude(z), decimals=1) == 3.0, "Something isn't quite right yet."
print("Nice, your magnitude function passed the basic sanity check!")

Nice, your magnitude function passed the basic sanity check!


Now, we will define a function to calculate the phase of the FT. For this we define the function `phase(ft)`, which should return the phase of the FT given as input.

Remember that the phase of a complex number $z$ is given by
$$\angle(z)=\arctan\left(\frac{\operatorname{Im}(z)}{\operatorname{Re}(z)}\right)\,.$$

For **0.5 points**, complete the function `phase(ft)` in the cell below according to the equation given above.

<div class="alert alert-info">

**Hint:** Use [`np.arctan2(x1, x2)`](https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html) to calculate the $\arctan\left(\frac{x_1}{x_2}\right)$. This function even chooses the corresponding quadrant for you, so you do not need to check for negative angles.
</div>

<div class="alert alert-warning">

**Beware:** Using `np.angle` in this exercise will give you **0 points**! Implement the function yourself.
</div>

In [164]:
%use sos
# Function that calculates the phase of complex numbers
def phase(ft):
    # Initialize output variable
    output = 0
    
    # YOUR CODE HERE
    output = np.arctan2(np.imag(ft), np.real(ft))
    return output

In [165]:
%use sos
# Let's do a sanity check
# The complex number used for the test which has a phase of pi/4
z = 1 + 1j
# Check that the magnitude function is correct
assert phase(z) == np.pi/4, "Something isn't quite right yet."
print("Great, your phase function passed the sanity check!")

Great, your phase function passed the sanity check!


Let's look at the resluts of the functions we just coded. For this we will apply the FT to the `car` image, calculate its magnitude and phase with the function you coded above and visualize the result as an image. Run the next cell to see the magnitude and phase of the `car` image.

In [166]:
%use sos
# Generate the FT of car
car_ft = fourier(car)
# Calculate the magnitude
car_ft_mag = magnitude(car_ft)
# Calculate the phase
car_ft_ph = phase(car_ft)
# Visualize the two together with the original image
plt.close('all')
ft_vis = viewer([car, car_ft_mag, car_ft_ph], title=['Car', 'Car FT magnitude', 'Car FT phase'], subplots=(1,3))

Button(description='Show Widgets', style=ButtonStyle())

### Multiple Choice Questions 
The following questions will test your understanding of the relationship of an image with its FT **magnitude**. The first two will ask about the image `car`, and the second two will ask about the image `pens`. Each MC question is worth **0.5 points**. As usual, we will include a cell for you to change your answer (in the variable `answer`) and a cell to check that you chose one of the possible answers. 

Run the next cell to visualize the images `pens` and `car` together with their FT magnitudes and answer the upcoming questions.

In [167]:
%use sos
# get the FT magnitudes of the images using the fourier and magnitude functions
car_ft  = magnitude(fourier(car))
pens_ft = magnitude(fourier(pens))

# define the lits of images and names
image_list = [car, car_ft, pens, pens_ft]
title_list = ['Car', 'FT of Car', 'Pens', 'FT of Pens']

# display results
plt.close('all')
ft_viewer = viewer(image_list, title=title_list, subplots = [2, 2], colorbar = True)

Button(description='Show Widgets', style=ButtonStyle())

* Q1: Where do the little stars at different distances from the center in the FT of `car` come from?
    1. From the contour of the car.
    2. From the driver.
    3. From the carpet under the car.
    4. From the details of the car (JAGUAR text, doors, steering wheel, etc).
    5. From the size of the image.

In the next cell, modify the variable `answer` to reflect your choice.

In [168]:
%use sos

# Assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [169]:
%use sos

assert answer in [1, 2, 3, 4, 5], 'Possible answers are 1, 2, 3, 4 and 5'


* Q2: Where do the two big intersecting lines in `car` come from? 
    1. From the contour of the car.
    2. From the driver.
    3. From the carpet under the car.
    4. From the details of the car (JAGUAR text, doors, steering wheel, etc).
    5. From the size of the image.

In the next cell, modify the variable `answer` to reflect your choice.

In [170]:
%use sos

# Assign your answer to this variable
answer = 1
# YOUR CODE HERE

In [171]:
%use sos

assert answer in [1, 2, 3, 4, 5], 'Valid answers are 1, 2, 3, 4, and 5'

* Q3: Why is there only one main line in the FT of `pens`, if there are two pens?
    1. Because the background is constant.
    2. Because the two pens are aligned.
    3. Because the two pens are close to each other.
    4. Because they are ballpoint pens and not fountain pens.

In the next cell, modify the variable `answer` to reflect your choice.

In [172]:
%use sos

# Assign your answer to this variable
answer = 2
# YOUR CODE HERE

In [173]:
%use sos

assert answer in [1, 2, 3, 4], 'Valid answers are 1, 2, 3 or 4'

* Q4: Why is the main line not aligned with the pens? 

    1. The main periodicity in the image is *perpendicular* to the pens because they are very similar.
    2. The main periodicity in the image is *parallel* to the pens because they have rough surfaces.
    3. The `viewer` rotates the FT.
    4. Numpy rotates the FT.

In the next cell, modify the variable `answer` to reflect your choice.

In [1]:
%use sos

# Assign your answer to this variable
answer = 1
# YOUR CODE HERE

In [2]:
%use sos

assert answer in [1, 2, 3, 4, 5], 'Valid answers are 1, 2, 3, 4 and 5'

## <a class="anchor"></a> 2.B. Reconstruction (2 Points)
[Back to index](#-Index)

As you learned in the course, it is possible to reconstruct an image from its Fourier transform by performing the inverse Fourier transform. In this next exercise we will investigate the role that the magnitude and phase of the FT has on the reconstruction of an image. For this we first need to create a function that reconstructs an image from its FT magnitude (in dB) and phase. Run the next cell to define the function `reconstruct(ft)` and make sure that you understand every line of the code.

In [176]:
%use sos
# Function that reconstructs an image from its FT magnitude (in dB) and phase
def reconstruct(mag, ph):
    # Since the magnitude is in dB we first need to convert it back
    mag = 10 ** (mag / 20)
    # Now we can restore the complex FT from the magnitude and phase using the polar representation
    ft = mag * np.exp(1j * ph)
    # Having the complex FT we can simply use the inverse_fourier function that we defined above to reconstruct the image
    return inverse_fourier(ft)

Let's see if the function works. Run the cell below to reconstruct the car image from its magnitude and phase, and visualize the result.

In [177]:
%use sos
# Reconstruct the car image
car_reconstructed = reconstruct(car_ft_mag, car_ft_ph)
# Display the result
plt.close('all')
ft_rec_vis = viewer([car, car_reconstructed], title=['Original car', 'Reconstructed car'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

Since we didn't make any changes to the FT before the reconstruction, the reconstructed image should be (almost) identical to the original image. 

For the next excercise we will use the `mandrill` image in addition to the car image. Run the next cell to visualize the `mandrill` image. This will help you to answer the upcoming questions.

In [178]:
%use sos
plt.close('all')
mandrill_vis = viewer(mandrill)

Button(description='Show Widgets', style=ButtonStyle())

Now lets see what happens if we use the magnitude of one image and the phase of another image to do the reconstruction. What do you think will happen? Run the cell below and observe the results. Try to make a conclusion on what type of information is stored in the phase of the FT.

In [179]:
%use sos
# Generate FT of the mandrill image
mandrill_ft = fourier(mandrill)
# Extract the magnitude and phase
mandrill_ft_mag = magnitude(mandrill_ft)
mandrill_ft_ph = phase(mandrill_ft)
# Reconstruct an image with the magnitude of car and phase of mandrill
car_mandrill = reconstruct(car_ft_mag, mandrill_ft_ph)
# Reconstruct an image with the magnitude of mandrill and phase of car
mandrill_car = reconstruct(mandrill_ft_mag, car_ft_ph)
# Visualize the results
plt.close('all')
rec_comp_vis = viewer([car_mandrill, mandrill_car], title=['Magn. = car, Phase = mandrill', 'Magn. = mandrill, Phase = car'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

### Multiple choice question

What type of information of the image is stored in the **phase** of the FT that is **not stored** in the magnitude? (**0.5 points**)

1. The spacial frequencies contained in the image.
2. The light intensity of each pixel.
3. The location and shape of objects in the image.

Modify the variable `answer`in the next cell to reflect your choice. The second cell is for you to check that you have entered a valid answer.

In [180]:
%use sos
# Assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [181]:
%use sos
# Sanity check
assert answer in [1, 2, 3], 'Valid answers are 1, 2 or 3'

### Reconstruction error

How many Fourier coefficients do we really need to keep to still have the basic information present in an image? Do any coefficients contribute the same? In order to answer this type of questions, it is good to define an objective metric of _how good_ a certain reconstruction is. Here, we will discuss the normalized mean square error (NMSE) to reconstruct an image $f$. This metric assesses how different an image $f$ is from its reconstruction $g$, but normalizes it by the total power of $f$. In particular, we have that if $K$ and $L$ are the number of columns and rows, respectively,

$$\operatorname{NMSE}_f(g) =  \frac{\sum_{k=1}^{K} \sum_{l=1}^L (g[k,l] - f[k,l])^2}{\sum_{m=1}^{K} \sum_{n=1}^L f[m,n]^2}\mbox{, and } \operatorname{NMSE}_f(g)~[\mathrm{dB}] = 10 \log_{10}\left(\operatorname{NMSE}_f(g)\right).$$

As we can see, the NMSE is often expressed in dB. This makes it easier for one to observe, for example, when the error has doubled ($+3~\mathrm{dB}$) or halved ($-3~\mathrm{dB}$) in plots.

For **0.5 points**, complete the function `nmse(f, g)` in the cell below according to the equation given above.
<div class="alert alert-info">

**Hint:** Use [`np.mean(x)`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) to calculate the mean value of `x`, or [`np.sum(x)`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) to calculate its sum through all axis.
</div>

In [182]:
%use sos
# Function that calculates the Normalized Mean Square Error in dB
def nmse(f, g):
    # Declare the output variable
    output = 0
    
    # YOUR CODE HERE
    output = 10*np.log10(np.sum((g-f)**2)/np.sum(f**2))
    # Return MSE
    return output

In [183]:
# Sanity check (do not worry about the divide by zero note)
assert nmse(impulse, impulse) == -np.infty, 'The error between two equal images should be zero. In log -infinity.'
# Check your function on the hrct image
assert nmse(impulse, 0) == 0, 'The error of any image and a zero-image should be 1. In log, 0.'
# Print victory message
print('Nice, your function seems to work! Do not worry about the divide by zero warning!')

<string>:6: RuntimeWarning: divide by zero encountered in log10


Nice, your function seems to work! Do not worry about the divide by zero warning!


### Fourier components

In this section we look into the reconstruction process of an image from part of its Fourier components. This touches on a topic that will continue to appear in IP1 and IP2: how much does a given transform compress an image? 

For a first practical approximation to the topic, we define the method `clip_fourier(img, precent)`. This method reconstructs an image for only `percent`$\%$ of its Fourier coefficients. If `largest=True`, only the largest are kept, while 
if `largest=False`, only all the rest are kept. This will illustrate the uneven distribution of information contained in the Fourier components.

Run the next cell to define the function `clip_fourier`.

In [184]:
%use sos

def clip_fourier(img, percent, largest=True, perc = True):
    # Get number of coefficients to keep
    if perc:
        # Extract from percentage
        n = np.round( np.prod(img.shape) * percent / 100 ).astype(np.int)
    else: 
        # Pass directly
        n = percent
    # Get ft of img
    img_ft = np.fft.fft2(img)
    # Get the threshold value. To do this, we order the Fourier coefficients 
    # from low to high and select the n-to-last ([-n]) coefficient
    threshold = np.sort(np.abs(img_ft.flatten()))[-n]
    if largest == True:
        # Get the inverse Fourier transform of the thresholded Fourier transform
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) >= threshold) * img_ft))
    else:
        # Get the inverse Fourier transform of the thresholded Fourier transform
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) < threshold) * img_ft))
    return clipped_ift

Let's use the error metric `nmse` defined before to illustrate the difference in information contained in the few largest Fourier components compared to the information contained in the rest. In the cell below we reconstruct the image `zebra` once using only the $50\%$ largest Fourier Components and once using the $50%$ largest components. Then we compare the reconstruction error by applying the `nmse` function defined above with both reconstructions. Run the cell below to see the different reconstruction errors. Play with the variable `percent` and see what happens.

In [185]:
%use sos
percent = 20
# First, reconstruct zebra using the largest components
zebra_largest = clip_fourier(zebra, percent)
# Reconstruct zebra using the smallest components
zebra_smallest = clip_fourier(zebra, percent, largest=False)
# Calculate the errors
error_l = nmse(zebra,zebra_largest )
error_s = nmse(zebra,zebra_smallest)
# Compare the error
print(f'The reconstruction error when using the {percent}% largest  components: NMSE = {error_l:.4f}')
print(f"The reconstruction error when using the {100 - percent}% smallest components: NMSE = {error_s:6.4f}")

The reconstruction error when using the 20% largest  components: NMSE = -2.3494
The reconstruction error when using the 80% smallest components: NMSE = 21.3549


In the long run, this type of characteristics of transforms are explored using graphs like the one below, where the NMSE can be seen as a function of the percentage of the largest coefficients kept.

In [186]:
plt.close("all")
# Create figure
fig = plt.figure(num=f"SCIPER: {uid}",figsize = (8, 5));
# Plot the NMSE vs kept coefficients curve
plt.plot( np.arange(.5,100,5), [nmse(zebra, clip_fourier(zebra, perc)) for perc in np.arange(.5,100,5)], 'bo-' );
# Labels and titles for clear plotting
plt.xlabel("Percentage of coefficients kept"); plt.ylabel("NMSE"); 
plt.xticks([0,20,40,60,80,100], [f"{perc}%" for perc in [0,20,40,60,80,100]]);
plt.title("Reconstructing zebra with the largest Fourier coefs.")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

Now we are going to create a widget to apply the function to an image and dinamically visualize its effect. 

We will define a slider to choose an integer `n` such that **the number largest coefficients kept is $2^n$**. 
<div class = "alert alert-info">
This is because the visual difference between the reconstructions is only apparent for percentages between $0\%$ and $2\%$, and very small steps would be needed. Keep in mind that you are not working with percentages anymore.
    
</div>We will also provide a checkbox to switch between the two modes of operation (keeping the largest, or keeping all the rest). Click the button `Apply` to apply `clip_fourier()` with the chosen parameter on the image. Run the next cell and click on `Extra Widgets` to use the widget. Explore the results carefuly.

In [187]:
%use sos

# Declare slider and checkbox
n_slider = widgets.IntSlider( value = 16, min = 0, max = 16, step = 1, description = 'n' )
checkbox = widgets.Checkbox(value=True, description='Use largest components')

# Declare btutton
button = widgets.Button(description = 'Apply')

# declare the button callback
def button_callback(image):
    n      = n_slider.value
    check  = checkbox.value
    output = clip_fourier(image, 2**n, largest = check, perc = False)
    return output

# visualize
plt.close('all')
cfourier_viewer = viewer(zebra, title = "Clipping the FT", new_widgets = [n_slider, checkbox, button], callbacks = [button_callback], widgets=True, normalize=True)

### Multiple Choice Question
Congratulations! You made it to the end of the notebook. Now you just need to answer these last two MCQ questions (**0.5 points** each).

* Q1: How many largest Fourier coefficients are required to start noting a zebra shape in the image?
    1. 1
    2. 8
    3. $2^8$
    4. $8^2$

Modify the variable `answer` to reflect your choice. The second cell will raise an error if you have not answered.

In [188]:
%use sos

# assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [189]:
%use sos

assert answer in [1, 2, 3, 4], 'Possible answers are 1, 2, 3 and 4'

* Q2: How is it possible to reconstruct the zebra from only periodic components if the zebra is not periodic (there is only one zebra in the image)?
    1. The black and white stripes in the zebra make it possible.
    2. The FT assumes that the image is periodic in space.
    3. The biggest components of the FT are non-periodic, to account for such features in an image.

Modify the variable `answer` to reflect your choice. The second cell will raise an error if you have not answered.

In [190]:
%use sos

# Assign your answer to this variable
answer = 2
# YOUR CODE HERE

In [191]:
%use sos

assert answer in [1, 2, 3], 'Possible answers are 1, 2, and 3'

<p><b>Congratulations on finishing Lab 1!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a>.
</p>

* Name it: *SCIPER_Pixel_Fourier.ipynb* (e.g. *308442_Pixel_Fourier.ipynb*)


<div class="alert alert-danger">
<h4>Feedback</h4>
    <p style="margin:4px;">
    This is the first edition of the image-processing laboratories using Jupyter Notebooks running on Noto. Do not leave before giving us your <a href="https://moodle.epfl.ch/mod/feedback/view.php?id=1097807">feedback here!</a></p>
</div>

